In [20]:
from pymongo import MongoClient

def insert_data():
    client = MongoClient('mongodb://localhost:27017/')
    db = client.local
 # 책 데이터 삽입
    books = [
        {"title": "Kafka on the Shore", "author": "Haruki Murakami", "year": 2002},
        {"title": "Norwegian Wood", "author": "Haruki Murakami", "year": 1987},
        {"title": "1Q84", "author": "Haruki Murakami", "year": 2009}
    ]
    db.books.insert_many(books)

    # 영화 데이터 삽입
    movies = [
        {"title": "Inception", "director": "Christopher Nolan", "year": 2010, "rating": 8.8},
        {"title": "Interstellar", "director": "Christopher Nolan", "year": 2014, "rating": 8.6},
        {"title": "The Dark Knight", "director": "Christopher Nolan", "year": 2008, "rating": 9.0}
    ]
    db.movies.insert_many(movies)

    # 사용자 행동 데이터 삽입
    user_actions = [
        {"user_id": 1, "action": "click", "timestamp": "2023-04-12T08:00:00Z"},
        {"user_id": 1, "action": "view", "timestamp": "2023-04-12T09:00:00Z"},
        {"user_id": 2, "action": "purchase", "timestamp": "2023-04-12T10:00:00Z"}
    ]
    db.user_actions.insert_many(user_actions)

    print("Data inserted successfully")
    client.close()

if __name__ == "__main__":
    insert_data()


from pymongo import MongoClient
from datetime import datetime

# 특정 장르의 책 찾기
def find_fantasy_books():
    client = MongoClient('mongodb://localhost:27017/')
    db = client.local
    results = db.books.find(
        {"genre":"fantasy"},
        {"title": 1, "author": 1, "_id": 0}
    )
    for book in results:
        print(book)
find_fantasy_books()

# 감독별 평균 영화 평점 계산
def avgerage_movie_rating():
    client = MongoClient('mongodb://localhost:27017/')
    db = client.local
    results = db.movies.aggregate([
        {"$group": {"_id":"$director","avgerage_movie_rating":{"$avg": "$rating"}}},    
        { "$sort": { "average_movie_rating": -1 } }
    ])
    for movie in results:
        print(movie)
avgerage_movie_rating()

# 특정 사용자의 최근 행동 조회
def recent_react():
    client = MongoClient('mongodb://localhost:27017/')
    db = client.local
    query = {"user_id":1}
    query2 = [("timestamp",-1)]
    results = db.user_actions.find(query).sort(query2).limit(5)

    for user_action in results:
        print(user_action)    
recent_react()

# 출판 연도별 책의 수 계산
def year_books():
    client = MongoClient('mongodb://localhost:27017/')
    db = client.local
    results = db.books.aggregate([
        {"$group":{
            "_id":"$year",
            "book_count":{"$sum":1}
        }},
        {"$sort":{"book_count":-1}}
    ])
    for book in results:
        print(book)
year_books()

# 특정 사용자의 행동 유형 업데이트
def update_user_action():
    client = MongoClient('mongodb://localhost:27017/')
    db = client.local
    update = db.user_actions
    date = datetime(2023,4,10)
    query = {
        "user_id": 1,
        "action": "view",
        "timestamp": { "$lt": date }  
    }
    new_action = {
        "$set": { "action": "seen" }
    }
    result = update.user_actions.update_many(query, new_action)
    print(f"New updated {result.modified_count}")

update_user_action()

Data inserted successfully
{'_id': 'Christopher Nolan', 'avgerage_movie_rating': 8.8}
{'_id': ObjectId('68219818ab57a6cfb90dad8f'), 'user_id': 1, 'action': 'seen', 'timestamp': datetime.datetime(2023, 4, 12, 9, 0)}
{'_id': ObjectId('68219818ab57a6cfb90dad8e'), 'user_id': 1, 'action': 'click', 'timestamp': datetime.datetime(2023, 4, 12, 8, 0)}
{'_id': ObjectId('68219bd5ab57a6cfb90dadad'), 'user_id': 1, 'action': 'view', 'timestamp': '2023-04-12T09:00:00Z'}
{'_id': ObjectId('68219bbdab57a6cfb90dada3'), 'user_id': 1, 'action': 'view', 'timestamp': '2023-04-12T09:00:00Z'}
{'_id': ObjectId('68219baeab57a6cfb90dad99'), 'user_id': 1, 'action': 'view', 'timestamp': '2023-04-12T09:00:00Z'}
{'_id': 2009, 'book_count': 51}
{'_id': 1987, 'book_count': 51}
{'_id': 2002, 'book_count': 51}
New updated 0
